In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
!pip install transformers

In [ ]:
# Import libraries

import numpy as np # linear algebra
import pandas as pd # data processing,

# Libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import plotly.express as px

# Import scikit_learn module for the algorithm/model: Linear Regression
from sklearn.linear_model import LogisticRegression
# Import scikit_learn module to split the dataset into train.test sub-datasets
from sklearn.model_selection import train_test_split
# Import scikit_learn module for k-fold cross validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# import the metrics class
from sklearn import metrics
# import stats for accuracy
import statsmodels.api as sm
# Instantiate LabelEncoder

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import joblib
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from transformers import pipeline
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
import plotly.graph_objects as go
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/jobs_train.csv')
#View the data
df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362685006848,COMP14,MANAGER,MASTERS,MATH,FINANCE,24,36,158
1,JOB1362685403468,COMP36,JANITOR,NONE,NONE,FINANCE,3,4,73
2,JOB1362684563560,COMP56,MANAGER,HIGH_SCHOOL,NONE,SERVICE,11,23,103
3,JOB1362684814664,COMP28,JANITOR,NONE,NONE,EDUCATION,22,42,63
4,JOB1362685123815,COMP48,CEO,DOCTORAL,NONE,OIL,15,66,129


## Exploratory data analysis (EDA)

In [ ]:
# data shape

df.shape

(900000, 9)

In [ ]:
# Describe the data

df.describe()

,yearsExperience,milesFromMetropolis,salary
count,900000.000000,900000.000000,900000.000000
mean,11.991183,49.525906,116.067520
std,7.211222,28.883348,38.717799
min,0.000000,0.000000,0.000000
25%,6.000000,25.000000,88.000000
50%,12.000000,50.000000,114.000000
75%,18.000000,75.000000,141.000000
max,24.000000,99.000000,301.000000


In [ ]:
# Find the duplicates

df.duplicated().sum()

0

In [ ]:
# Find null values

df.isnull().sum()

jobId                  0
companyId              0
jobType                0
degree                 0
major                  0
industry               0
yearsExperience        0
milesFromMetropolis    0
salary                 0
dtype: int64

In [ ]:
# get information about data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   jobId                900000 non-null  object
 1   companyId            900000 non-null  object
 2   jobType              900000 non-null  object
 3   degree               900000 non-null  object
 4   major                900000 non-null  object
 5   industry             900000 non-null  object
 6   yearsExperience      900000 non-null  int64 
 7   milesFromMetropolis  900000 non-null  int64 
 8   salary               900000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 61.8+ MB


In [ ]:
# Datatypes

df.dtypes

jobId                  object
companyId              object
jobType                object
degree                 object
major                  object
industry               object
yearsExperience         int64
milesFromMetropolis     int64
salary                  int64
dtype: object

In [ ]:
# Remove unwanted chars to convert to float in next step

df['jobId'] = df['jobId'].replace('JOB','', regex=True)


In [ ]:
# Convert object datatype from object to float

for columnName,columnData in df[['jobId','yearsExperience','salary','milesFromMetropolis']].items():
  df[columnName]=df[columnName].astype(float)

In [ ]:
# Datatypes after procesing

df.dtypes

jobId                  float64
companyId               object
jobType                 object
degree                  object
major                   object
industry                object
yearsExperience        float64
milesFromMetropolis    float64
salary                 float64
dtype: object

In [ ]:
# Unique values for numuric columns

for (columnName, columnData) in df.items():
        if pd.api.types.is_float_dtype(columnData):
          print('Column Name : ', columnName)
          print('Column unique values : ', df[columnName].unique())
          print(end="\n")

Column Name :  jobId
Column unique values :  [1.36268501e+12 1.36268540e+12 1.36268456e+12 ... 1.36268540e+12
 1.36268463e+12 1.36268492e+12]

Column Name :  yearsExperience
Column unique values :  [24.  3. 11. 22. 15.  1. 12. 13. 17. 10.  0. 16.  5. 14.  2.  9. 18. 19.
 21. 23.  8.  6. 20.  4.  7.]

Column Name :  milesFromMetropolis
Column unique values :  [36.  4. 23. 42. 66. 51. 21. 96.  9. 74. 26.  0. 90. 61. 10. 71. 49. 41.
 13. 95. 68. 67. 69. 44. 79. 37. 53.  1. 40. 63. 14. 57.  2. 98. 18. 91.
 38.  8. 56. 16. 19. 54. 81. 48. 39. 15.  3. 62. 24. 12. 20.  5. 86. 45.
 77. 28. 34. 30. 11. 55. 27. 99. 80. 47. 75. 76. 46. 89. 22. 87. 70. 59.
 65. 52. 43. 25. 33.  7. 94. 73. 72. 31. 83. 35. 85. 88. 84. 29.  6. 60.
 97. 82. 58. 92. 64. 50. 32. 93. 78. 17.]

Column Name :  salary
Column unique values :  [158.  73. 103.  63. 129. 169. 197. 110.  95.  81. 188. 231.  97. 181.
 139.  80.  87. 163. 101.  61.  66. 174.  68. 115.  57. 104. 170.  99.
 186. 183. 128. 152. 122.  74. 134.  79. 11

In [ ]:
# Unique values for String columns

string_cols=[]
for (columnName, columnData) in df.items():
        if pd.api.types.is_string_dtype(columnData):
          string_cols.append(columnName)
          print('Column Name : ', columnName)
          print('Column unique values : ', df[columnName].unique())
          print(df[columnName].value_counts())
          print(end="\n")

Column Name :  jobType
Column unique values :  ['MANAGER' 'JANITOR' 'CEO' 'CTO' 'JUNIOR' 'CFO' 'VICE_PRESIDENT' 'SENIOR']
SENIOR            113266
VICE_PRESIDENT    112755
CTO               112648
JANITOR           112427
CEO               112357
MANAGER           112356
JUNIOR            112196
CFO               111995
Name: jobType, dtype: int64

Column Name :  degree
Column unique values :  ['MASTERS' 'NONE' 'HIGH_SCHOOL' 'DOCTORAL' 'BACHELORS']
HIGH_SCHOOL    213205
NONE           213095
BACHELORS      158006
DOCTORAL       157930
MASTERS        157764
Name: degree, dtype: int64

Column Name :  major
Column unique values :  ['MATH' 'NONE' 'BIOLOGY' 'COMPSCI' 'CHEMISTRY' 'LITERATURE' 'BUSINESS'
 'PHYSICS' 'ENGINEERING']
NONE           478953
CHEMISTRY       53052
LITERATURE      52901
ENGINEERING     52799
BUSINESS        52750
PHYSICS         52570
BIOLOGY         52544
COMPSCI         52456
MATH            51975
Name: major, dtype: int64

Column Name :  industry
Column unique valu

In [ ]:
# check for trailing whitespaces in string column data

for i in string_cols:
    df[i] = df[i].str.strip()

# Data Visualiztion

##### jobType Column Visualization                

In [ ]:
import plotly.express as px
import seaborn as sns

# Assuming you have a DataFrame named 'df' with a column named 'jobType'

fig = px.histogram(df, x='jobType')

fig.update_traces(marker=dict(color='rgba(0, 123, 255, 0.7)'),
                  marker_line=dict(color='rgba(0, 123, 255, 1)', width=1))

fig.update_layout(title_text='Job Types Count', xaxis_title='Job Type', yaxis_title='Count')

fig.show()


##### Plot The top ten values of salary according to jobType

In [ ]:
fig = go.Figure()

grouped_df = df.groupby('jobType')['salary'].max().reset_index()
sorted_df = grouped_df.sort_values('salary', ascending=False).head(10)

fig.add_trace(go.Bar(
    x=sorted_df['jobType'],
    y=sorted_df['salary'],
    marker=dict(color='rgba(0, 123, 255, 0.7)'),
                  marker_line=dict(color='rgba(0, 123, 255, 1)', width=1)
))

fig.update_layout(
    xaxis=dict(title='jobType'),
    yaxis=dict(title='salary'),
    title='Top Ten Highest Salaries by jobType',
)

fig.show()


In [ ]:
# Group by jobType and calculate the mean salary
mean_salary = df.groupby(["jobType"]).agg({"salary": "mean"}).reset_index()
plt.figure(figsize=(13, 6))  # Adjust the width and height as desired

# Create the Seaborn bar plot
sns.barplot(x="jobType", y="salary", data=mean_salary)

# Set the title and axis labels
plt.title("Mean Salary Grouped by jobType")
plt.xlabel("jobType")
plt.ylabel("Mean Salary")

# Show the plot
plt.show()


### Major column visualization

In [ ]:
fig = px.histogram(df, x='major')

fig.update_traces(marker=dict(color='rgba(0, 123, 255, 0.7)'),
                  marker_line=dict(color='rgba(0, 123, 255, 1)', width=1))

fig.update_layout(title_text='Major Count', xaxis_title='Major', yaxis_title='Count')

fig.show()


In [ ]:
# Top Ten Highest Salaries by Major

fig = go.Figure()

grouped_df = df.groupby('major')['salary'].max().reset_index()
sorted_df = grouped_df.sort_values('salary', ascending=False).head(10)

fig.add_trace(go.Bar(
    x=sorted_df['major'],
    y=sorted_df['salary'],
    marker=dict(color='rgba(0, 123, 255, 0.7)'),
                  marker_line=dict(color='rgba(0, 123, 255, 1)', width=1),
))

fig.update_layout(
    xaxis=dict(title='Major'),
    yaxis=dict(title='salary'),
    title='Top Ten Highest Salaries by Major',
)

fig.show()


In [ ]:
# Group by Major and calculate the mean salary
mean_salary = df.groupby(["major"]).agg({"salary": "mean"}).reset_index()
plt.figure(figsize=(13, 6))  # Adjust the width and height as desired

# Create the Seaborn bar plot
sns.barplot(x="major", y="salary", data=mean_salary)

# Set the title and axis labels
plt.title("Mean Salary Grouped by major")
plt.xlabel("major")
plt.ylabel("Mean Salary")

# Show the plot
plt.show()


### Degree column visualization

In [ ]:
fig = px.histogram(df, x='degree')

fig.update_traces(marker=dict(color='rgba(0, 123, 255, 0.7)'),
                  marker_line=dict(color='rgba(0, 123, 255, 1)', width=1))

fig.update_layout(title_text='Degree Count', xaxis_title='Degree', yaxis_title='Count')

fig.show()


In [ ]:
# Top Ten Highest Salaries by Degree

import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Assuming you have a DataFrame named 'df' with columns 'jobType' and 'salary'

fig = go.Figure()

grouped_df = df.groupby('degree')['salary'].max().reset_index()
sorted_df = grouped_df.sort_values('salary', ascending=False).head(10)

fig.add_trace(go.Bar(
    x=sorted_df['degree'],
    y=sorted_df['salary'],
    marker=dict(color='rgba(0, 123, 255, 0.7)'),
                  marker_line=dict(color='rgba(0, 123, 255, 1)', width=1),
))

fig.update_layout(
    xaxis=dict(title='Degree'),
    yaxis=dict(title='salary'),
    title='Top Ten Highest Salaries by Degree',
)

fig.show()


In [ ]:
# Group by Degree and calculate the mean salary
mean_salary = df.groupby(["degree"]).agg({"salary": "mean"}).reset_index()
plt.figure(figsize=(13, 6))  # Adjust the width and height as desired

# Create the Seaborn bar plot
sns.barplot(x="degree", y="salary", data=mean_salary)

# Set the title and axis labels
plt.title("Mean Salary Grouped by degree")
plt.xlabel("degree")
plt.ylabel("Mean Salary")

# Show the plot
plt.show()


In [ ]:
fig = px.histogram(df, x='industry')

fig.update_traces(marker=dict(color='rgba(0, 123, 255, 0.7)'),
                  marker_line=dict(color='rgba(0, 123, 255, 1)', width=1))

fig.update_layout(title_text='Industry Count', xaxis_title='Industry', yaxis_title='Count')

fig.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Group by industry and find the maximum salary
grouped_df = df.groupby('industry')['salary'].max().reset_index()
sorted_df = grouped_df.sort_values('salary', ascending=False).head(10)

# Create the Seaborn bar plot
plt.figure(figsize=(10, 6))  # Adjust the width and height as desired
sns.barplot(x='industry', y='salary', data=sorted_df, palette='Blues_d')

# Set the title and axis labels
plt.title('Top Ten Highest Salaries by Industry')
plt.xlabel('Industry')
plt.ylabel('Salary')

# Rotate x-axis labels for better readability if needed
plt.xticks(rotation=45)

# Show the plot
plt.show()


In [ ]:
# Group by Industry and calculate the mean salary
mean_salary = df.groupby(["industry"]).agg({"salary": "mean"}).reset_index()
plt.figure(figsize=(13, 6))  # Adjust the width and height as desired

# Create the Seaborn bar plot
sns.barplot(x="industry", y="salary", data=mean_salary)

# Set the title and axis labels
plt.title("Mean Salary Grouped by Industry")
plt.xlabel("Industry")
plt.ylabel("Mean Salary")

# Show the plot
plt.show()


### yearExperience column *visualization*

In [ ]:
fig = px.scatter(df, x="yearsExperience", y="salary")
fig.show()

In [ ]:
sns.boxplot(data=df[['salary','yearsExperience','milesFromMetropolis']])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Group by yearsExperience and calculate the mean salary
mean_salary = df.groupby(["yearsExperience"]).agg({"salary": "mean"}).reset_index()
plt.figure(figsize=(13, 6))  # Adjust the width and height as desired

# Create the Seaborn bar plot
sns.barplot(x="yearsExperience", y="salary", data=mean_salary)

# Set the title and axis labels
plt.title("Mean Salary Grouped by Years of Experience")
plt.xlabel("Years of Experience")
plt.ylabel("Mean Salary")

# Show the plot
plt.show()


#### Correlation heatmab





In [ ]:
# correlation heatmab to show the correlation between columns

correlation_mat = df.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()

## Modeling

In [ ]:
df=df.drop(['jobId'], axis=1)

In [ ]:
# Define features and target
X = df.drop('salary', axis=1)
y = df['salary']
pipelines={}
df['companyId'] = df['companyId'].replace('COMP','', regex=True)

df['companyId'] = df['companyId'].astype(float)
# Define categorical and numerical features
categorical_features = ['jobType', 'degree', 'major', 'industry']
numerical_features   = ['companyId','yearsExperience' , 'milesFromMetropolis']

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Define models
models = [

    ('XGBoost', XGBRegressor()),
    ('LightGBM', LGBMRegressor()),
    ('Linear Regression', LinearRegression()),
]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

targetScaler=MinMaxScaler()
y_train=targetScaler.fit_transform(y_train.values.reshape(-1, 1))
y_test=targetScaler.transform(y_test.values.reshape(-1,1))
# To save mse for each model
model_errors = {}

# Preprocess, train and evaluate each model
for model_name, model in models:
    print(f'Training {model_name}...')
    # Create and fit the pipeline for each model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    pipeline.fit(X_train, y_train)
    pipelines.update({model_name: pipeline})
    # Make predictions
    y_pred = pipeline.predict(X_test)
    # Compute MSE
    mse = mean_squared_error(y_test, y_pred)
    # Save the error for each model
    model_errors[model_name] = mse
    print(f'{model_name} MSE: {mse}')

# Print the model with the least MSE
best_model = min(model_errors, key=model_errors.get)
print(f'\nThe best model is {best_model} with MSE: {model_errors[best_model]}')


## Save models

In [ ]:
pickle.dump(pipelines, open('/content/drive/MyDrive/models/Modell.pkl', 'wb'))


### Feature importance for each model

In [ ]:
# Get the feature names
feature_names = pipelines['XGBoost'][:-1].get_feature_names_out()

# Get the feature importances
feature_importances = pipelines['XGBoost'].steps[-1][-1].feature_importances_

# Create the horizontal bar plot
fig = go.Figure(data=go.Bar(
    x=feature_importances,
    y=feature_names,
    orientation='h'
))

# Update the layout
fig.update_layout(
    title='Feature Importances - XGBoost Model',
    xaxis_title='Importance',
    yaxis_title='Features'
)

# Show the plot
fig.show()


In [ ]:
# Get the feature names
feature_names = pipelines['LightGBM'][:-1].get_feature_names_out()
# Get the feature importances
feature_importances = pipelines['LightGBM'].steps[-1][-1].feature_importances_

# Create the horizontal bar plot
fig = go.Figure(data=go.Bar(
    x=feature_importances,
    y=feature_names,
    orientation='h'
))
# Update the layout
fig.update_layout(
    title='Feature Importances - LightGBM Model',
    xaxis_title='Importance',
    yaxis_title='Features'
)

# Show the plot
fig.show()


In [ ]:
import plotly.graph_objects as go

# Assuming pipelines is a dictionary containing the trained models
coefficients_list=[]
# Get the feature names
feature_names = pipelines['Linear Regression'][0].transformers_[1][1].get_feature_names_out()
# Get the feature importances for Linear Regression
if 'Linear Regression' in pipelines:
    lr_model = pipelines['Linear Regression'][1]
    coefficients = lr_model.coef_
    for i in coefficients:
      for j in i :
        coefficients_list.append(j)

    # Create the horizontal bar plot for Linear Regression
    fig_lr = go.Figure(data=go.Bar(
        x=coefficients_list,
        y=feature_names,
        orientation='h'
    ))

    # Update the layout for Linear Regression
    fig_lr.update_layout(
        title='Feature Importances - Linear Regression Model',
        xaxis_title='Coefficient',
        yaxis_title='Features'
    )

    # Show the plot for Linear Regression
    fig_lr.show()


## Deep Learning Model

In [ ]:
# Step 1: Perform one-hot encoding on categorical features (Role, Degree, Field, Specialization)
y=df['salary']

categorical_features = ['jobType',	'degree','major',	'industry']
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(df[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))


# Step 2: Scale numerical features (Experience1, Experience2) using StandardScaler


numerical_features = ['yearsExperience', 'milesFromMetropolis']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[numerical_features])
scaled_df = pd.DataFrame(scaled_features, columns=numerical_features)


# Step 3: Concatenate encoded and scaled features into a single DataFrame

processed_data = pd.concat([encoded_df, scaled_df], axis=1)


# Step 4: Split the data into training and testing sets for model training and evaluation

X_train, X_test, y_train, y_test = train_test_split(processed_data, y, test_size=0.2, random_state=42)

targetScaler=MinMaxScaler()
y_train=targetScaler.fit_transform(y_train.values.reshape(-1, 1))
y_test=targetScaler.transform(y_test.values.reshape(-1,1))
# Step 5: Build a deep learning model using Keras

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(processed_data.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error', metrics='accuracy')

model.fit(X_train, y_train, epochs=60, batch_size=32)

# Step 6: Evaluate the model on the test set and make predictions

loss = model.evaluate(X_test, y_test)
predictions = model.predict(X_test)

print("Test Loss:", loss)
print("Predictions:", predictions)


In [ ]:
# Get the weights of the model's layers
weights = model.get_weights()

# Calculate the feature importance
feature_importance = np.abs(weights[0]).sum(axis=1)

# Normalize feature importance to sum up to 1
feature_importance /= feature_importance.sum()

# Sort the features based on importance in descending order
sorted_features = X_train.columns[np.argsort(feature_importance)[::-1]]

# Create a bar plot of feature importance using Seaborn
plt.figure(figsize=(10, 6))  # Set the figure size
sns.barplot(x=feature_importance, y=sorted_features, orient='h')
plt.title("Feature Importance for Deep Learning Model")
plt.xlabel("Feature Importance")
plt.ylabel("Features")

# Show the plot
plt.show()

###  Save the deep learning model


In [ ]:
# # Save the model
model.save('/content/drive/MyDrive/models/deep_learning_model.h5')


# Second Section

In [ ]:
def predict_from_csv(csv_path, model_path):

    # Load the trained model
    with open(model_path, 'rb') as f:
       model = pickle.load(f)

    # Read the CSV file into a pandas DataFrame
    new_data=pd.read_csv(csv_path)

    # LightGBM is the best model to predict ,it has min loss value
    best_pipeline = pipelines['LightGBM']

    # Preprocess the new data
    new_data_processed = best_pipeline['preprocessor'].transform(new_data)

    # Make predictions using the best model
    predictions = best_pipeline['model'].predict(new_data_processed)

    # Print the predictions
    print("Predictions on new data:")
    print(predictions)
    print(end="\n")
    inverse_predictions = targetScaler.inverse_transform(predictions.reshape(-1, 1))
    print("Inverse transformed predictions:")
    print(inverse_predictions)

In [ ]:
predict_from_csv('/content/new_test_csv.csv','/content/drive/MyDrive/models/Modell.pkl')

Predictions on new data:
[0.56016418 0.25058354 0.34248355 0.26456544 0.52397236 0.53904221
 0.59249877 0.36238583 0.28580006]

Inverse transformed predictions:
[[168.60941859]
 [ 75.4256448 ]
 [103.08754821]
 [ 79.63419808]
 [157.71568015]
 [162.25170433]
 [178.34213014]
 [109.0781353 ]
 [ 86.02581661]]
